# map matching

In [1081]:
from rtree import index
import pandas as pd
from geopandas import GeoDataFrame
import heapq
import csv
import math
from shapely.geometry import LineString, Point
import time
import numpy as np

nodes = GeoDataFrame.from_file('.\\nodes.shp',encoding='utf-8')

nnum = nodes['osmid'].iloc[-1] + 1

edges=GeoDataFrame.from_file('.\\edges.shp',encoding='utf-8')

edges['len'] = [ edges.iloc[i]['geometry'].length for i in range(len(edges))]


In [1082]:
def geom_bbox(geom):
    y= [i[1] for i in geom]
    x= [i[0] for i in geom]
    return min(x), min(y), max(x), max(y)
def argmin(list_):
    f = lambda i : list_[i]
    idx = min(range(len(list_)), key=f)
    return idx, list_[idx]
def norm(diff):
    return math.sqrt((diff[0])**2 + (diff[1])**2)
#list -> str linestring
def toLs(data):
    #2차원 배열
    str_ = "LINESTRING("
    for lon,lat in data:
        str_+="{lon} {lat},".format(lon= lon,lat=lat)
    return str_[:-1] +")"
def frLs(wkt):
    return list(map(lambda a : list(map(float,a.split())), wkt[len("LINESTRING")+1:-1].split(',')))    
    

In [1083]:
class transNode:
    def __init__(self,eid,ep,pgeom,lamb,po):
        #tp,prev는 가장 높은 확률을 가지는 이전 노드
        self.eid = eid
        self.pgeom = pgeom
        self.lamb = lamb
        #self.ls = ls
        #self.lr = lr
        self.ep = ep
        self.tp = 0
        self.stackp = 0
        self.prevalue = -1e+10 # value가 음수가 나오는 경우도있음.
        self.preid = -1
        self.po =  po #Point 객체
    def printNode(self):
        print(f"eid:{self.eid}")
        print(f"stackp:{self.stackp}")
        print(f"ep:{self.ep}")
        print(f"tp:{self.tp}")
        print(f"prevalue:{self.prevalue}")#
       # print(f"lr:{self.lr}")
        #print(f"ls:{len(self.ls)}")
        if self.preid != -1: print(f"preid:{self.preid.eid}")
        else: print(f"preid:{self.preid}")

In [1084]:


#dataframe은 속도가 느림.
class Network:
    def __init__(self,edges,nnum):
        #rtree에 edges내용물 집어넣음
        
        self.rtree = index.Index()
        edges_content = [{'u':i[0],'v':i[1],'key':i[2],'len':i[-1],'geometry':i[-2]}for i in edges.to_numpy()]
        enum= len(edges_content)
        eidx = list(range(enum))
        edges_or = dict(zip(eidx,edges_content))
        self.enum = enum
        self.nnum = nnum  #새 node의 번호.

        #edges를 가지고 graph 관계 도출
        # nodes, edges 정보.
        #  {nodeid : 관련항들.

        edges_dic= dict()
        self.uvk_dic = dict()
        for i,e in enumerate(edges_or.values()):
            #e=edges.iloc[i]
            if (e['u'],e['v'],e['key'])in self.uvk_dic:
                edges_dic[i] = self.uvk_dic[(e['u'], e['v'],e['key'])]
                
            else:
                self.uvk_dic[(e['u'], e['v'],e['key'])] = i
                self.uvk_dic[(e['v'], e['u'],e['key'])] = i
                edges_dic[i] = i
                
        
        #edges to index
        #self.uvk_dic = uvk_dic
        
        self.nodes= dict()
        self.edges = dict()
        self.edges_or = dict()
        
        # edges: {uid: { vid : {key : {length:.. , geometry,}}}
        for i,e in enumerate(edges_or.values()):
            
            if edges_dic[i] < i: 
                continue
            self.edges_or[i] = e
            #e = edges.iloc[i]
            
            #rtree에 추가
            coords = np.array(list(e['geometry'].coords))
            e_bbox = e['geometry'].bounds
            self.rtree.insert(i,e_bbox,obj=e['geometry'])
            
            #nodes 추가
            if e['u'] not in self.nodes:
                self.nodes[e['u']] = {'y': coords[0][1], 'x':coords[0][0]}
            if e['v'] not in self.nodes:
                self.nodes[e['v']] = {'y': coords[-1][1], 'x':coords[-1][0]}
            
            #edges 추가
            
            if e['u'] not in self.edges:
                self.edges[e['u']] = dict()
            
            if e['v'] not in self.edges[e['u']]:
                self.edges[e['u']][e['v']]= dict()
            #self.edges[e['u']][e['v']][e['key']] = i
            self.edges[e['u']][e['v']][e['key']] = {'length':e['len']}
                
            if e['v'] not in self.edges:
                self.edges[e['v']] = dict()
        
            if e['u'] not in self.edges[e['v']]:
                self.edges[e['v']][e['u']]= dict()
            self.edges[e['v']][e['u']][e['key']] = {'length':e['len']}
            
    def intersect(self, region):
        #rtree에 geom 겹치는 부분이 있는지 물어보는 쿼리
        #left down right up
        return self.rtree.intersection(region)
    def add_node(self,point):
        #point : np.array([경도,위도])
        self.nodes[self.nnum] = {'y':point[1],'x':point[0]}
        nid = self.nnum
        self.nnum += 1
        return nid
    def add_edge(self,u,v,geom):
        #geom ; linestring
        #key 결정
        key = 0
        while (u,v,key) in self.uvk_dic:
            key += 1
        #uvk_dic
        self.uvk_dic[(u,v,key)] = self.enum
        self.uvk_dic[(v,u,key)] = self.enum
        #edges_or
        self.edges_or[self.enum] = {'u':u,'v':v,'key':key,'len':geom.length,
                                   'geometry':geom}
        #edges
        if u not in self.edges:
            self.edges[u] = dict()
        if v not in self.edges[u]:
            self.edges[u][v]= dict()
        self.edges[u][v][key] = self.enum
        
        
        if v not in self.edges:
            self.edges[v] = dict()
        if u not in self.edges[v]:
            self.edges[v][u]= dict()
        self.edges[v][u][key] = self.enum
        #rtree
        self.rtree.insert(self.enum,geom.bounds,obj=geom )
        self.enum += 1

    def remove_edge(self,eid):
        #uvk_dic, edges, edges_or, rtree
        #edges_or 삭제
        e = self.edges_or.pop(eid)
        u = e['u']
        v = e['v']
        key = e['key']
        geom = e['geometry']
        #rtree 삭제
        self.rtree.delete(eid,geom.bounds)
        #edges
        del(self.edges[u][v][key])
        del(self.edges[v][u][key])
        #uvk_dic
        del(self.uvk_dic[(u,v,key)]) #삭제
        del(self.uvk_dic[(v,u,key)]) #삭제
        
        return u,v,geom
        #self.enum -= 1 
    def singleDikstra(self,src,radius,wr):
        #nidx에 해당하는 노드로 다익스트라 
        #source,target, dist, next_v, next_e, prev_v
        
        cdict= {src:0} # 최소 거리가 결정된 node들
        que = []
        
        #초기화
        f = lambda i: edges[i]
        for trg , items in list(self.edges[src].items()):
            
            #target,next_v,next_e,prev_v
            edges = [i['length'] for i in list(items.values())]
            key,dist=argmin(edges)
            #key = min(range(len(edges)), key=f)
            if (src,trg,key) not in self.uvk_dic: next_e = self.uvk_dic[(trg,src,key)]
            else: next_e = self.uvk_dic[(src,trg,key)]
            heapq.heappush(que,(dist,[trg,trg,next_e,src]))
        #print("initque",que)
        
        while que:

            #cur : 최단거리가 결정된 좌표.
            cur_dist,info = heapq.heappop(que) 
            if info[0] in cdict:
                continue
            else:
                cdict[info[0]]=cur_dist
                wr.writerow([src,info[0],cur_dist,info[1],info[2],info[3]])
                for trg , items in list(self.edges[info[0]].items()):
                    #print("cdict:",cdict)
                    #upper bound radius 
                    trg_node= [self.nodes[trg]['y'], self.nodes[trg]['x']]
                    src_node = [self.nodes[src]['y'], self.nodes[trg]['x']]
                    diff = [src_node[0] - trg_node[0], src_node[1] - trg_node[1]]
                    if norm(diff) > radius:
                        continue
                
                    #info: [target,next_v,next_e,prev_v]
                    edges = [i['length'] for i in list(items.values())]
                    key,dist = argmin(edges)
                    heapq.heappush(que,(cur_dist+dist,[trg,info[1],info[2],info[0]]))
            #print(cdict)
   
    def searchKnn(self, traj, k,gps_error,r=0.0005):
        #traj :Linestring(str)
        #left down right up
        
        #candidate node 만들기
        traj_list =frLs(traj)
        
        cds=[]
        unmatch = []
        #Point(traj_list[0])
        for num,row in enumerate(traj_list):
            po = Point(row)
            #bound = po.buffer(r) #seg 단위로 할 의미가없음
            all_eids=list(self.rtree.intersection((row[0] - r ,row[1]-r,row[0]+r,row[1]+r)))
            #print(len(all_eids))
            #if not all_eids:
            #    unmatch.append(num)
            #    continue
            #numpy로 broadcasting가능하다면 하는게 더 빠를듯.
            all_dists=[po.distance(self.edges_or[i]['geometry']) for i in all_eids]

            #all_dists=[1/po.distance(self.edges_or[i]['geometry']) for i in all_eids]
            #all_dists_sum = sum(all_dists)
            eids=list(zip(all_eids,all_dists))
            eids.sort(key=lambda a:a[1]) #reverse
            #if num == 0:
            #    print(all_dists_sum)
            #    print(eids)
            #eids : 최종 k개 후보
            #eids = list(self.rtree.nearest((row[0] - r ,row[1]-r,row[0]+r,row[1]+r), k))
            cd=[]
            len_sum = sum([self.edges_or[eid]['len'] for eid,dist in eids[:k]])
            for eid,dist in eids[:k]:
                line = self.edges_or[eid]['geometry']
                lamb = line.project(po)
                pgeom = line.interpolate(lamb)
                
                #ep = dist/all_dists_sum # ep 
                ep = self.emission(gps_error,dist)
                #ep = self.logep(gps_error,dist)
                
                #lr = self.edges_or[eid]['len'] / len_sum
                #ls = bound.intersection(line)
                #if isinstance(ls,LineString) or isinstance(ls,Point):
                    #교점이 point인 경우도 고려
                #    ls = [ls]
                #else:
                #    ls = list(ls)
                #lsarr= list(map(self.ls_to_numpy,ls))
                # 교점이있다면 lsarr의 첫번째멤버로 []가 올수없음
                #교점이없는경우 
                #if lsarr[0].size == 0:
                #    #print("no intersection")
                #    continue
                
                cd.append(transNode(eid=eid,ep=ep,pgeom=pgeom,lamb=lamb,po=Point(row)))#ep 수정해야해
            if cd:
                cds.append(cd)
            else:
                unmatch.append(num)
                
        return cds, unmatch
    
    def ls_to_numpy(self,ls):
        #ls : point or ls
        lsnp = np.array(list(ls.coords))
        if len(lsnp) == 1:
            return np.vstack([lsnp,lsnp])
        else:
            return lsnp
        
    def emission(self,gps_error,dist):
        #broadcasting 활용하면 numpy 수학 이용하는게 빠름.
        return math.exp(-(dist**2)/(2*(gps_error**2)))/(math.sqrt(2*math.pi)*gps_error)
    def logep(self, gps_error,dist):
        return -(dist**2) / (2 * (gps_error ** 2)) 
    

In [1085]:

class FMM:
    def __init__(self,cinfo,network):
        #FMM 모델만들기
        #ubodt : src,trg index 되어있는 df
        #network : Network
        ubodt  = dict()
        dist = cinfo['dist'].to_numpy()
        prev = cinfo['prev_v'].to_numpy()
        nextv = cinfo['next_v'].to_numpy()
        nexte = cinfo['next_e'].to_numpy()
        key = cinfo.index.to_numpy()
        for k,d,p,n,e in zip(key,dist,prev,nextv,nexte):
            #next_e 일단 고려 X
            ubodt[k] = {'dist':d, "prev":p,"nextv":n,"nexte":e}       
        self.ng = network
        self.ubodt = ubodt
        
        
    def ep(self,gps_error,dist):
        #broadcasting 활용하면 numpy 수학 이용하는게 빠름.
        return math.exp(-(dist**2)/(2*(gps_error**2)))/(math.sqrt(2*math.pi)*gps_error)
    def logep(self, gps_error,dist):
        #혹시물라서 추가
        return -(dist**2) / (2 * (gps_error ** 2))
    def match_wkt(self,wkt,fmm_config):
        #wkt : linestring(str)
        #fmm_config : r, k, gps_error
        k = fmm_config['k']
        r = fmm_config['r']
        gps_error = fmm_config['gps_error']
        #tp = fmm_config['tp']
        wkt_list = np.array(frLs(wkt))
        #init node
        cds,unmatch = self.ng.searchKnn(wkt,k=k,gps_error=gps_error,r=r)
        
        if unmatch:
            return {'unmatch':unmatch}
        #for fcd in cds[0]:
        #    fcd.stackp = 2 * math.log(fcd.ep) # 2배 늘려줌
        #nodes stackp
        
        #P(ti | ti-1)을 제대로 구함.
        # vieterbi 에서 max, argmax값만 안바뀌면 된다는점을 이용. overflow 해결.
        for cid in range(len(cds)-1):
            #print(f"-------{cid}")
            ipo = wkt_list[cid]
            jpo = wkt_list[cid+1]
            edist = norm(ipo-jpo) # np.linalg.norm 보다 일차원 배열엔 효율적
            #print(f"cid:{cid,cid+1}, edist:{edist}")
            for nj,j in enumerate(cds[cid+1]):
                #cur node
                for ni,i in enumerate(cds[cid]):
                    #prev node
                    #print(f"eids:{i.eid},{j.eid}")
                    #print()
                    #if (i.eid, j.eid) in self.ubodt:
                    #    ijtp = tp[self.ubodt[(i.eid,j.eid)]]
                    #else:
                    #    ijtp = 1e-10
                    
                    "tp"
                    ijtp,nodes = self.trans(i,j,edist)
                    #ijtp,nodes=self.accurate_trans(0.001,i,j,edist)
                    
                    "ep"
                    #points = [Point(ipo), Point(jpo)]
                    #ijep = self.accurate_emiss(i,j,nodes,points,gps_error)
                    
                    #iep =  i.ep
                    jep =  j.ep
                    
                    #jep = self.approx_ep(j,jpo,0.0005,gps_error)
                   
                    "cal allp"
                    allp = math.log(ijtp) + math.log(jep) #+ math.log(iep) 
                    #allp = math.log(ijtp) + math.log(ijep)
                    value = i.stackp + allp
                    if value > j.prevalue : 
                        j.preid= i
                        j.tp = ijtp
                        j.prevalue = value
                        j.stackp = allp
                        #j.ep = ijep
                
        
        #backtrack
        LNode=max(cds[-1],key = lambda a:a.stackp)
        opath=[]
        #pgeom = 'LineString('
        pgeom = []
        #i = -1
        while LNode.preid != -1 :
            opath.append(LNode.eid)
            pgeom.append(LNode.pgeom)
            #i -= 1 
            LNode = LNode.preid
        opath.append(LNode.eid)
        pgeom.append(LNode.pgeom)
        #pgeom += str(LNode.pgeom) + ')'
        opath.reverse()
        pgeom.reverse()
        return {
            'opath' :opath,
            'pgeom': pgeom
        }
    def match_file(self,filedata,fmm_config):
        #filedata : LS array
        #pandas로 만들어 저장.
        #fileindex = file.to_numpy()[:,0]
        result = []
        oid = []
        uid = []
        for num,wkt in enumerate(filedata):
            #print(num)
            mr = self.match_wkt(wkt,fmm_config)
            if 'opath' in mr: oid.append(num)
            else : uid.append(num)
            result.append(mr)
        return np.array(result),oid, uid

    def spdist(self,i,j):
        #i, j : transNode
        nodes = []
        ie = self.ng.edges_or[i.eid]
        je = self.ng.edges_or[j.eid]
        ipo= np.array(i.pgeom)
        jpo = np.array(j.pgeom)
        ivlist = [ie['u'],ie['v']] #geom : u->v순서
        jvlist = [je['u'],je['v']]
        middle_dist = np.inf # pgeom sp_dist
        if i.eid == j.eid:
            middle_dist = norm(ipo - jpo)
        else:
            # 두 seg의 네방형(u,v 조합) 모두 고려. 가장 짧은 거리 반환
            for iv in ivlist:
                inode = self.ng.nodes[iv]
                icoord = np.array([inode['x'],inode['y']])
                for jv in jvlist:
                    jnode = self.ng.nodes[jv]
                    jcoord = np.array([jnode['x'],jnode['y']])
                    dist = norm(jpo - jcoord) + norm(ipo - icoord)
                    if iv != jv:
                        dist += self.ubodt[(iv,jv)]['dist']
                    if dist < middle_dist:
                        middle_dist = dist
                        nodes = (iv,jv)
        return middle_dist,nodes
        
    def trans(self,i,j,edist):
        #check redist 0
        #ver : 분수, exp
        
        middle_dist,nodes= self.spdist(i,j)
                   
        #point trans : 
        """tp = 0 
        if middle_dist == 0 and edist == 0:
            tp = 1
        else:
            tp =  min(middle_dist, edist)/max(middle_dist, edist)
        if tp == 0:
            tp = 1e-10
        return tp,nodes"""
        
        beta = 0.0005
        diff = - abs(edist - middle_dist)/beta
        return (1/beta) * math.exp(diff ), nodes
        
    
    def accurate_trans(self,std_len,i,j,edist):

        sp_dist,nodes = self.spdist(i,j)
        #중심포인트에서 매칭
        left = abs(std_len - sp_dist)
        right = std_len + sp_dist
        #print(f"sp_dist:{sp_dist} , left:{left}, right:{right}")
        if sp_dist > std_len:
            return self.cal_trans(left,right,edist),nodes
        else:
            return self.cal_trans(0,right,edist) + self.cal_trans(0,left,edist),nodes
    def cal_trans(self,left,right,edist,beta = 0.0005):
        #print(f"debug left:{left}")
        start = left - edist
        end = right - edist
        
        stp = self.tp(abs(start))
        etp = self.tp(abs(end))
        #print(f"start,end:{start,end}")
        #print(f"stp,etp:{stp,etp}")
        if start * end < 0:
            # start ,end 부호다름
            zero = 1
            #print(stp,etp)
            #print(f"p : {2 - stp - etp}")
            return 2 - stp - etp
        else:
            # 적분식은 거리 min - max 형태
            #print(f"p : {abs(stp - etp)}")
            return abs(stp-etp)
            
        
            
    def tp(self,diff,beta=0.0005): 
        #beta = 0.0005
        x = - abs(diff)/beta
        return  math.exp(x)

    def connect_edge(self,ieid,jeid,nodes,std_len=0.0005):
        if ieid == jeid:
            #같은 eid에 매칭
            return list(self.ng.edges_or[ieid]['geometry'].coords)
        trg = nodes[1]
        src = temp = nodes[0]
        #ij 간선 연결 : 
        ie = self.ng.edges_or[ieid]
        je = self.ng.edges_or[jeid]
        start = True
        end = True
        if src != trg:
            row = self.ubodt[(src,trg)]
            if row['nextv'] in [ie['u'],ie['v']]:
                start = False
            if row['prev'] in [je['u'], je['v']]:
                end = False
        edge =[]
        i = 0
        while temp != trg:
            row = self.ubodt[(temp,trg)]
            eid = self.ng.edges_or[row['nexte']]
            coords = list(eid['geometry'].coords)
            if eid['u'] == temp:
                #정방향
                edge += coords[1:] #겹치는 경우 막기위해
            else:
                edge += coords[::-1][1:] #reverse?
            temp = row['nextv']
 
            i+=1
        coords = list(ie['geometry'].coords)
        if ie['u'] == src:
            coords = coords[::-1] 
        if start:
            edge = coords + edge
        else:
            edge = [coords[-1]] + edge

        coords = list(je['geometry'].coords) 
        if je['u'] != trg:
            coords = coords[::-1] 
        if end:
            edge = edge + coords[1:]
        #end좌표는 이미 있으니 딱히 필요없음.

        return edge
    def new_point(self,u,v,std_len):
        # u - v방향
        difx = (u[0] - v[0])
        dify = (u[1] - v[1])
        diag = norm([difx,dify])
        cos = difx/diag
        sin = dify/diag
        return (u[0] + std_len * cos, u[1] + std_len*sin)
        #깔끔한 코드를위해 첨부터 확장시킴.
    def seg_integral_ep(self,u,v,po,gps_error):
        #oo : Point
        #print(type(po))
        ep = 0
        seg = LineString([u,v])
        std = po.distance(seg)
        left = po.distance(u)
        right = po.distance(v)
        epstd = self.ep(gps_error,std)
        ep += (left-std) * (self.ep(gps_error,left) + epstd) / 2
        ep += (right - std) * (self.ep(gps_error,right) + epstd) / 2
        return ep
    
    def integral_ep(self,line,std_len,inode,po,gps_error ):
        #po : inode po : Point object
        #line : list
        ls = LineString(line)
        lamb = ls.project(Point(inode.pgeom)) #pgeom point객체로
        
        start = ls.interpolate(lamb - std_len)
        end = ls.interpolate(lamb + std_len)
        
        start_lamb = round(lamb - std_len,7)
        end_lamb = round(lamb + std_len,7)
        
        #po = inode.po
        curlen = prelen= ep = 0
        i = 0
        #start 지역 뽑기
        while i < len(line) - 1: 
            u = line[i]
            v = line[i+1]
            curlen += norm([u[0] - v[0] , u[1] - v[1]])
            if prelen <= start_lamb and start_lamb <= curlen:
                prelen = curlen
                break
            prelen = curlen
            i += 1
        
        #시작적분값 :
        ep += self.seg_integral_ep(start,Point(line[i+1]),po,gps_error)
        i += 1
        #적분계산
        while i <len(line) - 1:
            u = line[i]
            v = line[i+1]
    
            curlen += norm([u[0] - v[0], u[1] - v[1]])
            if prelen < end_lamb and end_lamb < curlen:
                break
            ep += self.seg_integral_ep(Point(u),Point(v),po,gps_error)
            prelen = curlen
            i += 1
        #마지막적분
        ep += self.seg_integral_ep(Point(line[i]),end,po,gps_error)
        return ep
    def accurate_emiss(self, inode, jnode,nodes,points, gps_error):
        #일단은 log ep 이용 X
        #points : list - Point(gps_pint)
        #nodes : list - connecting nodes
        ipo = points[0]
        jpo = points[1]
        std_len = 0.0005
        line = self.connect_edge(inode.eid,jnode.eid,nodes,std_len)
        
        #간선확장 : 양 끝을 기준으로 std_len씩 증가.
        left_point= self.new_point(line[0],line[1],std_len)
        right_point = self.new_point(line[-1],line[-2],std_len)
        line.insert(0,left_point)
        line.append(right_point)
 
        #적분계산 : 
        iep = self.integral_ep(line,std_len,inode,ipo,gps_error)
        jep = self.integral_ep(line,std_len,jnode,jpo,gps_error)
        
        return iep + jep
        #lamb를 가지고 확장해야하나 조사.
    def approx_ep(self, inode,po, std_len,gps_error=0.0005):
        
        edge = self.ng.edges_or[inode.eid]['geometry']
        coords = list(ie.coords)
        prev=cur = 0
        lamb = round(inode.lamb,7)
        for i in range(len(coords)-1):
            u= coords[i]
            v = coords[i+1]
            cur += norm([u[0] - u[1], v[0]-v[1]])
            if round(prev,7) < lamb and lamb <round(cur,7):
                break
            prev = cur
        newls = [coords[i] , coords[i+1]]
        newls.insert(0,self.new_point(newls[0],newls[1],std_len))
        newls.append(self.new_point(newls[1],newls[0],std_len))
        newls = LineString(newls)
        
        
        newlamb = newls.project(Point(inode.pgeom))
        bd1 = newls.interpolate(newlamb + std_len) 
        bd2 = newls.interpolate(newlamb - std_len)
        newpro = newls.interpolate(newlamb)
        
        std = round(newpro.distance(Point(po)),7)
        right = round(bd1.distance(Point(po)),7)
        left = round(bd2.distance(Point(po)),7)
        
        print(f"std:{std},left:{left},right{right}")
        stdep = self.ep(gps_error,std)
        #print(stdep)
        ep = (left - std )* (self.ep(gps_error,left)+ stdep) / 2
        #print(ep)
        ep += (right - std) * (self.ep(gps_error,right) + stdep) /2
        return ep


In [1086]:
newtj = pd.read_csv('.\\newtj.csv')
newtj = newtj.to_numpy()

In [1087]:
ng = Network(edges,nnum)

ubodt = pd.read_csv('.\\ubodt.csv')

ubodt= ubodt.set_index(['src','trg'])

model=FMM(ubodt,ng)

fmm_config = {'k':8,'r':0.0005,'gps_error':0.0005,'beta':0.003}

In [1108]:
import folium

all_map = folium.Map(location=[37.545585700351,126.913333897367],zoom_start=16,max_zoom=26,prefer_canvas=True)

n = 14
for i,v in enumerate(frLs(newtj[n][0])):
    folium.CircleMarker(
    location=[v[1],v[0]],
    radius=5,
    popup=i,
    color="gray",
    fill=True,
    fill_color="gray",
    ).add_to(all_map)
wkt_list = np.array(frLs(newtj[n][0]))
result = model.match_wkt(toLs(frLs(newtj[n][0])[:]),fmm_config)
eids = result['opath']
"""for i,pv in enumerate(result['pgeom']):
    v= list(pv.coords)[0]
    folium.CircleMarker(
    location=[v[1],v[0]],
    radius=5,
    popup=i,
    fill=True,
    fill_color="blue",
    color = "red"
    ).add_to(all_map)"""
for num,enum in enumerate(eids):
    ie = ng.edges_or[enum]['geometry']
    coords = list(map(lambda a:[a[1],a[0]], list(ie.coords)))
    folium.PolyLine(coords,
                    color='gray',
                    popup = enum
                   ).add_to(all_map)    

v = frLs(newtj[n][0])[14]
"""
folium.Circle(
location=[v[1],v[0]],
radius=50,
popup=i,
color="#3186cc",
fill=True,
fill_color="blue",
).add_to(all_map)
"""
all_map

In [1092]:
n = 14
for i,v in enumerate(frLs(newtj[n][0])):
    folium.CircleMarker(
    location=[v[1],v[0]],
    radius=5,
    popup=i,
    color="green",
    fill=True,
    fill_color="green",
    ).add_to(all_map)
wkt_list = np.array(frLs(newtj[n][0]))
result = model.match_wkt(toLs(frLs(newtj[n][0])[:]),fmm_config)
eids = result['opath']
"""for i,pv in enumerate(result['pgeom']):
    v= list(pv.coords)[0]
    folium.CircleMarker(
    location=[v[1],v[0]],
    radius=5,
    popup=i,
    fill=True,
    fill_color="blue",
    color = "red"
    ).add_to(all_map)"""
for num,enum in enumerate(eids):
    ie = ng.edges_or[enum]['geometry']
    coords = list(map(lambda a:[a[1],a[0]], list(ie.coords)))
    folium.PolyLine(coords,
                    color='green',
                    popup = enum
                   ).add_to(all_map)    

In [1093]:
all_map

In [15]:
#alltr 되도록 수정필요 0830
start = time.time()
result,oid,uid = model.match_file(newtj[:,0],fmm_config)
end = time.time()
#시간 대략 2분(20000개 데이터)
#속도 저하의 원인? trans?
print("time: ",end - start)

time:  44.332520484924316


{'opath': [41,
  41,
  41,
  41,
  40,
  39,
  39,
  39,
  39,
  39,
  39,
  354,
  354,
  354,
  354,
  354,
  285,
  285,
  285,
  286,
  286,
  286,
  286,
  286,
  2134,
  2134,
  2134,
  282,
  284,
  2665,
  284,
  2665,
  2665,
  2665,
  2665,
  2665,
  2665,
  2665,
  2667,
  2665,
  2667,
  2667,
  2667,
  2667,
  2667,
  2667,
  2667,
  2667,
  2667,
  294,
  2667,
  2667,
  2667,
  2667,
  2667,
  294,
  294,
  2667,
  2667,
  294,
  294,
  2667,
  2667],
 'pgeom': [<shapely.geometry.point.Point at 0x23057673d08>,
  <shapely.geometry.point.Point at 0x23057884f88>]}

# Node2vec

In [42]:
from torch_geometric.nn import Node2Vec
from torch_geometric.data import Data
import os.path as osp
import torch
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
from torch_geometric.datasets import Planetoid
from tqdm.notebook import tqdm
from torch_geometric.datasets import TUDataset

In [22]:
ng.edges

{436718644: {5215341444: {0: {'length': 0.00027908516621395437}},
  5215341441: {0: {'length': 0.0001835425291305809}},
  5215341440: {0: {'length': 0.00016713434715297714}}},
 5215341444: {436718644: {0: {'length': 0.00027908516621395437}},
  436835498: {0: {'length': 0.0005848622476074979}},
  3074324024: {0: {'length': 0.00028212628377784226}},
  5215341445: {0: {'length': 0.00022688402323716044}}},
 5215341441: {436718644: {0: {'length': 0.0001835425291305809}},
  5215341440: {0: {'length': 0.00012543360911510183}},
  5215341445: {0: {'length': 0.00026514043448330163}},
  3078076792: {0: {'length': 0.0005029105089399906}}},
 436718653: {3034111472: {0: {'length': 0.0004983292586242226}},
  0: {0: {'length': 0.0006136801385873328}},
  3034111465: {0: {'length': 0.0007318177368761464}}},
 3034111472: {436718653: {0: {'length': 0.0004983292586242226}},
  58: {0: {'length': 0.0005139585856705047}},
  114: {0: {'length': 0.0005327208607541353}},
  6673585414: {0: {'length': 0.0005634209

In [32]:
tempedges = list(ng.edges_or.values())

In [61]:
eids = list(ng.edges_or.keys())

1619

In [75]:
# 엣지들의 이름을 0~ 3038이라고 붙이고 노드로 간주
#이러한 노드끼리 이어져 있는 관계를 저장
edgeArr=[]

for i in eids:
    for j in eids:
        if i != j:
            if (ng.edges_or[i]['v'] ==  ng.edges_or[j]['u']):
                edgeArr.append([i,j])
            if (ng.edges_or[i]['u'] ==  ng.edges_or[j]['v']):
                edgeArr.append([j,i])
            

In [110]:
# 엣지들의 이름을 0~ 3038이라고 붙이고 노드로 간주
#이러한 노드끼리 이어져 있는 관계를 저장
# edges_or의 키로 하려고했는데 엠베딩 벡터 수가 이상해진다;
tempegdes = list(ng.edges_or.values())
edgeArr=[]

for i in range(len(tempegdes)):
    for j in range(len(tempegdes)):
        if i != j:
            if (tempegdes[i]['v'] ==  tempegdes[j]['u']):
                edgeArr.append([i,j])
            if (tempegdes[i]['u'] ==  tempegdes[j]['v']):
                edgeArr.append([j,i])
            

In [111]:
edge_index = torch.tensor(edgeArr)
x = torch.tensor([[x] for x in range(len(tempedges))])
#x = torch.tensor([[i] for i in list(ng.edges_or.keys())])

1619

In [113]:
data = Data(x=x, edge_index=edge_index.t().contiguous())

In [114]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [131]:
data.edge_index

tensor([[1384,    0, 1384,  ..., 1618, 1618, 1617],
        [   0, 1388,    1,  ...,  620,  621, 1618]])

In [236]:
embed_dim = 32

In [115]:
model = Node2Vec(data.edge_index, embedding_dim=embed_dim, walk_length=20,
             context_size=10, walks_per_node=10,
             num_negative_samples=1, p=1, q=1, sparse=True).to(device)

loader = model.loader(batch_size=128, shuffle=True, num_workers=0)
optimizer = torch.optim.SparseAdam(list(model.parameters()), lr=0.01)

In [116]:
def train():
    model.train()
    total_loss = 0
    for pos_rw, neg_rw in loader:
        optimizer.zero_grad()
        loss = model.loss(pos_rw.to(device), neg_rw.to(device))
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    return total_loss / len(loader)

In [117]:

for epoch in range(100):
    loss = train()
    #acc = test()
    if epoch % 10 == 0:
        print(f'Epoch: {epoch:02d}, Loss: {loss:.4f}')

Epoch: 00, Loss: 2.5665
Epoch: 10, Loss: 0.9401
Epoch: 20, Loss: 0.7884
Epoch: 30, Loss: 0.7588
Epoch: 40, Loss: 0.7478
Epoch: 50, Loss: 0.7431
Epoch: 60, Loss: 0.7406
Epoch: 70, Loss: 0.7389
Epoch: 80, Loss: 0.7384
Epoch: 90, Loss: 0.7377


In [142]:
z = model(torch.arange(data.num_nodes, device=device))
# from tensor to numpy
emb_32 = z.detach().cpu().numpy()

In [150]:
tempedges

[{'u': 436718644,
  'v': 5215341444,
  'key': 0,
  'len': 0.00027908516621395437,
  'geometry': <shapely.geometry.linestring.LineString at 0x2306156cac8>},
 {'u': 436718644,
  'v': 5215341441,
  'key': 0,
  'len': 0.0001835425291305809,
  'geometry': <shapely.geometry.linestring.LineString at 0x2306156f248>},
 {'u': 436718653,
  'v': 3034111472,
  'key': 0,
  'len': 0.0004983292586242226,
  'geometry': <shapely.geometry.linestring.LineString at 0x2306156f688>},
 {'u': 436718653,
  'v': 0,
  'key': 0,
  'len': 0.0006136801385873328,
  'geometry': <shapely.geometry.linestring.LineString at 0x2306156fb88>},
 {'u': 436835481,
  'v': 3869600353,
  'key': 0,
  'len': 0.0004031500961095731,
  'geometry': <shapely.geometry.linestring.LineString at 0x23061572308>},
 {'u': 436835481,
  'v': 1830499931,
  'key': 0,
  'len': 0.00017379369954301205,
  'geometry': <shapely.geometry.linestring.LineString at 0x23061572748>},
 {'u': 436835481,
  'v': 1,
  'key': 0,
  'len': 0.00047486626538828785,
  'g

In [152]:
embedding_idx = [i for i in range(len(tempedges))]

In [180]:
eids = list(ng.edges_or.keys())

In [196]:
eids[1317
]

2134

In [197]:
#eids -> embeid 관계 나타내는 dic
eids_dic = dict()
for eid, emid in zip(eids,embedding_idx):
    eids_dic[eid] = emid

1317

In [126]:
len(x)

1619

In [95]:
len(ng.edges_or.values())

1619

In [133]:
data.edge_index.t()

tensor([[1384,    0],
        [   0, 1388],
        [1384,    1],
        ...,
        [1618,  620],
        [1618,  621],
        [1617, 1618]])

In [ ]:
def make_cpath():
    

# skip gram

In [210]:
import torch
from torch import nn
import torch.optim as optim
import torch.nn.functional as F

In [199]:
#data 만들기

def skip_gram_data(tr, i, window_size = 2):
    #dataset 구성
    #tr. 간선집합 : list
    #i : 중심간선
    point = tr[i]
    left = tr[max(i - window_size,0):i]
    right = tr[i:i+1+window_size]
    center_id = eids_dic[point]
    #[[center, context]]
    return [(center_id,eids_dic[cid]) for cid in left] + \
            [(center_id,eids_dic[cid]) for cid in right]

def get_all_batches(tr,window_size= 2):
    #tr : list집합. 간선 번호 시퀀스
    data = []
    for i in range(len(tr)):
        data += skip_gram_data(tr,i,window_size = window_size)
    return data


def make_temp_cpath(tr):
    #연속으로 중복된 간선만 삭제.
    #result 원소 하나만 cpath 만들어주기
    eids = tr['opath']
    temp_cpath = [eids[0]]
    for i in range(len(eids) - 1):
        if eids[i] != eids[i+1]:
            temp_cpath += [eids[i+1]]
    return temp_cpath

def make_temp_cpath_result(result):
    cpath_result = []
    for i in result:
        #print(i)
        cpath_result.append(make_temp_cpath(i))
    return cpath_result

In [515]:
"raw tr"
#train_data_set = result[oid][:350]
#test_data_set = result[oid][350:]

"cpath tr"
train_data_set = make_temp_cpath_result(result[oid][:350])
test_data_set = make_temp_cpath_result(result[oid][350:])

In [772]:
#make dataset
ws = 20 # window size
dataset = []

#raw tr
"""for tr in train_data_set:
    dataset += get_all_batches(tr['opath'],window_size=ws)"""

#cpath tr
for tr in train_data_set:
    dataset += get_all_batches(tr,window_size= ws)

In [773]:
dataset = np.array(dataset)

In [778]:
edges_len

1619

In [776]:
len(test_data_set)

143

In [777]:
len(train_data_set) + len(test_data_set)

493

In [445]:
#embedding vecdtor scailing : 별 차이는 없는듯함

from sklearn.preprocessing import RobustScaler

# 변형 객체 생성
robust_scaler = RobustScaler()

# 훈련데이터의 모수 분포 저장
robust_scaler.fit(emb_32)

# 훈련 데이터 스케일링
scaled_emb_32 = robust_scaler.transform(emb_32)


In [990]:
class SkipGram(nn.Module):
    def __init__(self,embed_dim,edges_len):
        #embed_vector: 엠베딩 벡터 정보
        #edges_dic : eid -> emid 매칭 딕셔너리
        super(SkipGram, self).__init__()
        seq = []
        hidden_size = 2
        for i in range(hidden_size):
            seq += [nn.Linear(embed_dim,embed_dim),nn.BatchNorm1d(embed_dim),nn.LeakyReLU(0.2)] #
        seq += [nn.Linear(embed_dim, edges_len)] #output layer
        self.seq = nn.Sequential(*seq)
    def forward(self,x):
        # x : batchsize x embed_dim: [centers...]
        return self.seq(x)
    def test_one(self,x):
        #한 중심 간선이왔을때 다른 간선의 softmax값 출력
        #x : 1 x embeid_dim : [center]
        return F.softmax(self.seq(x))

array([  36,   36,   36,   36,   36,   36,   36,   36,   36,   36,   36,
         36,   36,   36,   36,   36,   36,   35,   35,   35,   35,   35,
         34,   34,   34,   34,   34,   34,   34,   34,   34,   34,   34,
         34,   34,   34,   34,   34,   34,   34,   34,   34,   34,   34,
         34,   34,   34,   34,   34,   34,   34,   34,  279,  279,  279,
        279,  279,  279,  279,  279,  279,  279,  279,  279,  279,  279,
        279,  279,  279,  279,  279,  279,  279,  279,  279,  279,  279,
        223,  223,  223,  223,  223,  223,  223,  223,  223,  223,  223,
        223,  223,  223,  223,  224,  224,  224,  224,  224,  224,  224,
        224,  224,  224,  224,  224,  224,  224,  224,  224,  224,  224,
        224,  224,  224,  224,  224,  224,  224, 1317, 1317, 1317, 1317,
       1317, 1317, 1317, 1317, 1317, 1317, 1317, 1317, 1317, 1317, 1317,
        221,  221,  221,  221,  221,  222,  222,  222,  222,  222, 1525,
       1525, 1525, 1525, 1525,  222,  222,  222,  2

In [528]:
def get_batch_id(data,batch_size,edges_len):
    idx = np.random.choice(len(data),batch_size)
    batch = data[idx]
    #source, target embedid
    return batch[:,0], batch[:,1]

In [992]:
batch_size = 256
epoch = len(dataset) // batch_size
edges_len = len(emb_32)

skipmodel = SkipGram(embed_dim,edges_len).to(device)
optimizer = optim.Adam(skipmodel.parameters(), lr =0.01)


In [982]:
#train

skipmodel.train()
for i in range(epoch):
    src, trg= get_batch_id(dataset,batch_size,edges_len)
    trg= torch.tensor(trg)
    trg= trg.type(torch.long)
    #src_embed = torch.tensor(([emb_32[i] for i in src]))
    src_embed = torch.tensor(([scaled_emb_32[i] for i in src])) #scaled
    out = skipmodel(src_embed.to(device)) #forward
    loss = F.cross_entropy(out,trg.to(device))
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    if i % 100 == 0:
        print(f"epoch : {i}, loss:{loss}")

epoch : 0, loss:7.5142436027526855
epoch : 100, loss:4.613085746765137
epoch : 200, loss:4.2413716316223145
epoch : 300, loss:4.427311420440674


[41, 40, 39, 354, 285, 286, 2134, 282, 284, 2665, 284, 2665, 2667, 2665, 2667, 294, 2667, 294, 2667, 294, 2667]
[2930, 1666]
[2930, 1666]
[2138, 284, 2665, 2667, 294, 2667, 1667, 1665, 1667, 1666, 1667, 1666]
[1666, 2930, 1666]
[1666, 2930, 1666]
[1667, 1665, 1667, 1666]
[2667]
[284, 2152, 1666, 2930, 1666, 2930, 2675, 1112, 2675, 1112, 2675, 2673, 291, 42, 43, 47]
[219, 217, 1110, 951, 1115, 951, 1115, 951, 1110, 951, 1110]
[47, 43, 46, 43, 42, 290, 302, 305, 301, 302, 303, 332]
[334, 933, 2188, 404, 53, 2188, 404, 2184, 933, 2684, 2185, 2684, 53, 933, 334, 933, 334, 933, 334, 933, 2684, 933]
[933, 334, 332, 303, 302, 301, 748, 749, 301, 2128, 2131, 288, 285, 354]
[348, 356, 358, 323, 1869, 1871, 348, 358, 1887, 355, 285, 354, 285, 288, 2131, 748, 301, 290, 42, 43, 47, 219, 220, 952, 723, 724, 756, 730, 731, 729, 115, 116, 119, 125, 126, 68, 70, 68, 70, 875, 2780, 875, 2780, 875, 2780]
[69]
[39, 8, 39, 354, 285, 286, 2134, 282, 284, 2665, 2667, 294, 293, 2203, 2839, 1115, 2839, 1115, 

In [971]:
contain384

[]

In [819]:
result[13]['opath']

[933,
 933,
 933,
 933,
 933,
 933,
 933,
 933,
 933,
 933,
 933,
 933,
 933,
 334,
 332,
 332,
 332,
 332,
 332,
 332,
 332,
 303,
 303,
 303,
 303,
 303,
 303,
 303,
 302,
 301,
 301,
 748,
 749,
 301,
 2128,
 2131,
 2131,
 2131,
 2131,
 2131,
 288,
 288,
 288,
 288,
 288,
 288,
 285,
 285,
 285,
 285,
 285,
 285,
 354,
 354,
 354,
 354,
 354,
 354]

In [1000]:
# 해당 도로를 가진 Tr이 존재하니?
contain = []
for n,i in enumerate(train_data_set):
    #print(i)
    if 0 in i:
        contain.append(n)
print(contain)

[191]


tensor(0.0007, device='cuda:0', grad_fn=<SelectBackward>)

In [1109]:
key_array = np.array(list(ng.edges_or.keys()))
top = 30
#test
skipmodel.eval()

center_id = 348
test_id = eids_dic[center_id]
testx = torch.tensor([scaled_emb_32[test_id]])
prob = skipmodel.test_one(testx.to(device))[0]
#testx = torch.tensor(emb_32[test_id])
#prob = skipmodel.test_one(testx.to(device))
top_edge = torch.argsort(prob,descending=True)[:top].cpu()
key_array[top_edge]

C:\Users\yeons\anaconda3\envs\gnn\lib\site-packages\ipykernel_launcher.py:18: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


array([ 285,  348,  358, 1869,  288, 2667,  323,   43, 2131,   42,  219,
        356,  748,  355,  290,   47,  354,  286,  220,  301, 2665, 2134,
       1887,  294,  282,  723, 1871,   40,  217,  281])

In [1111]:
#visual:  간선 u,v 확인
import folium

all_map = folium.Map(location=[37.545585700351,126.913333897367],zoom_start=16,max_zoom=26,prefer_canvas=True)

#eids= [0,2292]

for num,enum in enumerate(key_array[top_edge]):
    einfo = ng.edges_or[enum]
    ie = einfo['geometry']
    unum = einfo['u']
    vnum = einfo['v']
    u = ng.nodes[unum]
    v = ng.nodes[vnum]
    coords = list(map(lambda a:[a[1],a[0]], list(ie.coords)))
    folium.PolyLine(coords,
                    color='red',
                    popup = enum
                   ).add_to(all_map)   
einfo = ng.edges_or[center_id]
ie = einfo['geometry']
coords = list(map(lambda a:[a[1],a[0]], list(ie.coords)))
folium.PolyLine(coords,
                color='blue',
                popup = center_id
               ).add_to(all_map) 
all_map

In [1075]:

top = 10
per_list= []
for i in range(edges_len):
    testx = torch.tensor([scaled_emb_32[i]])
    prob = skipmodel.test_one(testx.to(device))[0]
    top_edge = torch.argsort(prob,descending=True)[:top].cpu()
    real_top_edge_id = key_array[top_edge]
    per_list.append (list(real_top_edge_id))

C:\Users\yeons\anaconda3\envs\gnn\lib\site-packages\ipykernel_launcher.py:18: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


In [1056]:
check_list = np.zeros((len(per_list),top),dtype=np.int32).tolist()

In [1057]:
all_list = np.zeros(len(per_list),dtype=np.int32).tolist()

In [1028]:
#per_list, check_list,all_list : idx로 접근

In [1031]:
test_data_set

[[275, 281, 282, 284, 2665, 2667, 2665, 2930, 1666],
 [1666],
 [2938, 294, 292, 2203, 2930, 1666],
 [2203, 294, 293, 294, 2202, 1115, 2839, 2203, 2839, 293],
 [1666, 2930, 1666, 2930, 1666, 2930, 1666, 2930, 1666, 2930, 1666],
 [1881],
 [1877,
  1881,
  39,
  354,
  1899,
  354,
  285,
  286,
  287,
  286,
  2134,
  2135,
  282,
  2665,
  2667,
  2665,
  2667],
 [2667, 2665, 284, 282, 281, 2152, 275],
 [275,
  277,
  278,
  275,
  277,
  275,
  278,
  275,
  281,
  282,
  284,
  2665,
  2667,
  294,
  2667],
 [2667,
  2665,
  284,
  282,
  281,
  275,
  278,
  1158,
  263,
  1158,
  263,
  265,
  264,
  265,
  264,
  265,
  264,
  1158,
  1157,
  263,
  1158,
  278,
  275,
  281,
  282,
  284,
  2665,
  2667,
  294,
  292,
  293,
  2930,
  1666,
  2930,
  293,
  294,
  2202,
  294],
 [2202, 292, 2202, 2203],
 [2203,
  2839,
  2203,
  294,
  2667,
  294,
  2667,
  294,
  292,
  294,
  292,
  294,
  2667,
  294,
  2667,
  294,
  2667,
  294,
  2667,
  294,
  2667,
  294,
  293,
  2202,
 

In [1032]:
all_list

array([0, 0, 0, ..., 0, 0, 0])

In [1076]:
#실험
# test 에서 중심 노드를 하나 선택. 선택한 노드의 좌,우 주변에 
# skip gram top 10이 몇개있는지 조사.

ws = 20

skipmodel.eval()
#exist = 0
#all_context_len = 0
exist = 0
nonexist = 0
for test_tr in test_data_set:
    for num,center_id in enumerate(test_tr):
        #test
        #print(num)
        idx = eids_dic[center_id]
        #all_list[idx] += 1 #전체 경우 하나 올림
        left = test_tr[max(num - ws,0):num]
        right = test_tr[num:num+1+ws]

        context = left + right
        #all_context_len += top
        #all_context_len += len(context)
        #print(context)
        nonexist += 1
        for rn,rid in enumerate(per_list[idx]):
            if rid in context:
                exist += 1
                nonexist -= 1
                break
                #print("?")
                #context안에 rid가 하나라도 있는경우 + 1
                #check_list[idx][rn] += 1
                



114

In [1080]:
exist/nonexist

12.894736842105264

In [1063]:
for n,i in enumerate(all_list):
    if i != 0:
        print(check_list[i])

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 1, 0, 0, 2, 1, 1, 0]
[0, 0, 0, 1, 0, 0, 2, 1, 1, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 1, 1, 1, 1]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0

In [1064]:
all_list

[0,
 0,
 0,
 0,
 0,
 2,
 5,
 5,
 3,
 2,
 1,
 4,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 2,
 1,
 2,
 1,
 1,
 1,
 0,
 2,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 2,
 0,
 1,
 1,
 1,
 1,
 4,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 9,
 26,
 21,
 0,
 0,
 2,
 0,
 1,
 95,
 80,
 83,
 86,
 37,
 46,
 28,
 36,
 3,
 12,
 7,
 2,
 1,
 8,
 19,
 42,
 116,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 

In [953]:
cal_idx_list= list(set(center_list))

In [944]:
list(per_dic.values())

[[354, 0, 9, 17, 341, 39, 339, 11, 286, 281],
 [354, 286, 39, 9, 285, 341, 11, 0, 339, 2134],
 [464, 1362, 476, 845, 2614, 466, 465, 2884, 63, 1483],
 [341, 281, 286, 354, 294, 9, 11, 39, 0, 2667],
 [4, 2665, 354, 284, 39, 10, 8, 2119, 2667, 7],
 [354, 39, 2134, 268, 286, 41],
 [4, 1667, 2309, 1662, 2940, 1666, 2938],
 [294, 39, 354, 10, 285, 2134],
 [2667, 294, 355, 284],
 [1666, 2665, 2930, 2667, 284, 294],
 [284, 2667, 1666, 7, 4],
 [294, 2667, 4, 292],
 [1666, 2930, 2203, 1667, 2839, 2932, 2938, 1665, 293, 2667],
 [2930, 723, 2932, 294, 2839, 724, 2203, 1115, 2776, 759],
 [2667, 1666, 285, 2665, 2930, 354, 294, 286, 1158, 2134],
 [724, 723, 2776, 294, 93, 2839, 952, 2930, 2203, 2932],
 [354, 39, 286, 11, 2134, 341, 294, 8, 17, 285],
 [286, 354, 281, 2134, 285, 341, 2667, 339, 39, 282],
 [354, 286, 281, 341, 0, 285, 2134, 39, 339, 282],
 [69, 68, 78, 724, 116, 70, 1941, 759, 723, 125],
 [69, 68, 70, 855, 854, 860, 2415, 2388, 852, 862],
 [276, 278, 277, 281, 1662, 275, 280, 2940, 36

In [956]:
all_prob

78.60000000000002

In [954]:
all_prob = 0
for i in cal_idx_list:
    all_prob += (top - len(per_dic[i]))/top
    

In [955]:
all_prob/len(cal_idx_list)

0.431868131868132

In [928]:
cal_idx_list

[5,
 6,
 7,
 8,
 9,
 10,
 11,
 2571,
 2590,
 39,
 40,
 41,
 42,
 43,
 44,
 2087,
 2090,
 47,
 2106,
 2109,
 67,
 2115,
 2126,
 79,
 592,
 593,
 594,
 595,
 2132,
 2131,
 2134,
 2135,
 1108,
 1106,
 1112,
 1115,
 1627,
 1110,
 2142,
 95,
 608,
 609,
 2144,
 2659,
 2143,
 2149,
 2152,
 2665,
 1641,
 2667,
 2153,
 2158,
 2160,
 2673,
 2675,
 1662,
 1663,
 1664,
 1665,
 1666,
 1667,
 1157,
 1158,
 1670,
 1674,
 2192,
 2193,
 1682,
 1684,
 2202,
 2203,
 2246,
 2249,
 204,
 205,
 211,
 723,
 213,
 215,
 2776,
 217,
 218,
 219,
 2780,
 2784,
 226,
 227,
 748,
 749,
 750,
 756,
 245,
 757,
 759,
 763,
 2811,
 765,
 766,
 768,
 769,
 258,
 2309,
 263,
 264,
 265,
 2827,
 1292,
 269,
 1293,
 274,
 275,
 276,
 277,
 278,
 2839,
 280,
 281,
 282,
 2843,
 284,
 285,
 286,
 287,
 288,
 290,
 291,
 292,
 293,
 294,
 1828,
 831,
 832,
 836,
 837,
 838,
 849,
 338,
 851,
 337,
 1877,
 854,
 1881,
 2393,
 857,
 862,
 864,
 354,
 870,
 871,
 872,
 361,
 2922,
 1899,
 875,
 876,
 2924,
 2926,
 1905,
 2930

In [919]:
per_dic

{0: [354, 0, 9, 17, 341, 39, 339, 11, 286, 281],
 1: [354, 286, 39, 9, 285, 341, 11, 0, 339, 2134],
 2: [464, 1362, 476, 845, 2614, 466, 465, 2884, 63, 1483],
 3: [341, 281, 286, 354, 294, 9, 11, 39, 0, 2667],
 4: [4, 2665, 354, 284, 39, 10, 8, 2119, 2667, 7],
 5: [354, 39, 2134, 268, 286, 41],
 6: [4, 1667, 2309, 1662, 2940, 1666, 2938],
 7: [294, 39, 354, 10, 285, 2134],
 8: [2667, 294, 355, 284],
 9: [1666, 2665, 2930, 2667, 284, 294],
 10: [284, 2667, 1666, 7, 4],
 11: [294, 2667, 4, 292],
 12: [1666, 2930, 2203, 1667, 2839, 2932, 2938, 1665, 293, 2667],
 13: [2930, 723, 2932, 294, 2839, 724, 2203, 1115, 2776, 759],
 14: [2667, 1666, 285, 2665, 2930, 354, 294, 286, 1158, 2134],
 15: [724, 723, 2776, 294, 93, 2839, 952, 2930, 2203, 2932],
 16: [354, 39, 286, 11, 2134, 341, 294, 8, 17, 285],
 17: [286, 354, 281, 2134, 285, 341, 2667, 339, 39, 282],
 18: [354, 286, 281, 341, 0, 285, 2134, 39, 339, 282],
 19: [69, 68, 78, 724, 116, 70, 1941, 759, 723, 125],
 20: [69, 68, 70, 855, 854, 

In [880]:
exist

6935

In [881]:
exist/all_context_len

0.4378156565656566

In [758]:
#모델 세이브
import os
torch.save(skipmodel.state_dict(),'skipgram.pth')

In [993]:
#모벨 로드
skipmodel.load_state_dict(torch.load('skipgram.pth'))

<All keys matched successfully>

In [ ]:

#scailing : 일단 없어도 될듯? 값 범위도 그리 튀지않고 안정적임. 결과보고 하는게 좋다면 추가
#dataset 구성 : train, test